In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import math

from matplotlib import pyplot as plt
%matplotlib inline

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

pd.set_option('display.float_format', lambda x: '%.3f' % x)

import warnings
warnings.filterwarnings('ignore')

In [ ]:
data_raw = pd.read_excel("data/default of credit card clients.xls",header=1)

In [ ]:
data = data_raw.copy()
print(data.shape)
data.head(50)

## Check for missingness

In [ ]:
data.isnull().sum()

In [ ]:
data.describe()

In [ ]:
sns.histplot(data['default payment next month'])

In [ ]:
sns.distplot(data.AGE)

# There are some strange spikes around the decade intervals

In [ ]:
# data dictionary: https://archive.ics.uci.edu/ml/datasets/default+of+credit+card+clients#

In [ ]:
sex = {1: "male", 2: "female"}
marital_status = {1: "married", 2: "single", 3: "other", 0: "0?"}
education = {1: "graduate_school", 2: "unversity", 3: "high_school", 4: "other", 5: "5?", 6: "6?", 0: "?"}


In [ ]:
sns.histplot(data.SEX.replace(sex))

In [ ]:
sns.histplot(data.MARRIAGE.replace(marital_status).astype(str))

In [ ]:
sns.histplot(data.EDUCATION.replace(education).astype(str))
plt.xticks(rotation=45)

## Check bill variables

In [ ]:
# Variables PAY_0 to PAY_6 represent the payment status for the months
# from April to September 2005
# They use the following coding schema:
# -2 = no bill (deduced)
# -1 = duly paid
# n = payment dealy for n months

In [ ]:
temp = data.filter(items=['PAY_0','PAY_2','PAY_3','PAY_4','PAY_5','PAY_6'])
sns.catplot(x="variable", y="value",
            kind="boxen", data=pd.melt(temp))
#plt.show()

In [ ]:
sns.histplot(data.PAY_0.astype(str))

In [ ]:
# Strangely, there is no PAY_1 variable

In [ ]:
# Bill amounts
temp = data.filter(items=['BILL_AMT1','BILL_AMT2','BILL_AMT3','BILL_AMT4','BILL_AMT5','BILL_AMT6'])
sns.catplot(x="variable", y="value",
            kind="violin", data=pd.melt(temp))
plt.ticklabel_format(style='plain', axis='y')
#plt.show()

In [ ]:
sns.distplot(data.BILL_AMT1)
plt.ticklabel_format(style='plain', axis='x')

In [ ]:
# Zoomed in on the range [0,100000]
bill_vars = ['BILL_AMT1','BILL_AMT2','BILL_AMT3','BILL_AMT4','BILL_AMT5','BILL_AMT6']
temp = data.filter(items=bill_vars)
q = ""
for i in range(0,len(bill_vars)):
    q = q + "0<=" + bill_vars[i] + "<=100000"
    if i <= len(bill_vars)-2:
        q = q + " and "
temp = temp.query(q)
sns.catplot(x="variable", y="value",
            kind="violin", data=pd.melt(temp))
plt.ticklabel_format(style='plain', axis='y')

In [ ]:
# Inspect negative bills
bill_vars = ['BILL_AMT1','BILL_AMT2','BILL_AMT3','BILL_AMT4','BILL_AMT5','BILL_AMT6']
temp = data.filter(items=bill_vars)
q = ""
for i in range(0,len(bill_vars)):
    q = q + "" + bill_vars[i] + "<0"
    if i <= len(bill_vars)-2:
        q = q + " and "
temp = temp.query(q)
sns.catplot(x="variable", y="value",
            kind="boxen", data=pd.melt(temp))
plt.ticklabel_format(style='plain', axis='y')

In [ ]:
# Number of negative bills
print("Count, mean of negative bills")
bill_vars = ['BILL_AMT1','BILL_AMT2','BILL_AMT3','BILL_AMT4','BILL_AMT5','BILL_AMT6']
for b in bill_vars:
    print(b, data.loc[data[b]<0].shape[0], data.loc[data[b]<0][b].mean())

In [ ]:
# Payment amounts
payment_vars = ['PAY_AMT1','PAY_AMT2','PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6']
temp = data.filter(items=payment_vars)
sns.catplot(x="variable", y="value",
            kind="violin", data=pd.melt(temp))
plt.ticklabel_format(style='plain', axis='y')
#plt.show()

In [ ]:
# Zoomed in on the range [0,k]
k=15000
payment_vars = ['PAY_AMT1','PAY_AMT2','PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6']
temp = data.filter(items=payment_vars)
q = ""
for i in range(0,len(payment_vars)):
    q = q + "0<=" + payment_vars[i] + "<=" + str(k)
    if i <= len(payment_vars)-2:
        q = q + " and "
temp = temp.query(q)
sns.catplot(x="variable", y="value",
            kind="violin", data=pd.melt(temp))
plt.ticklabel_format(style='plain', axis='y')

In [ ]:
# Logical check for cases where bill paid is more than bill amount
bill_vars = ['BILL_AMT1','BILL_AMT2','BILL_AMT3','BILL_AMT4','BILL_AMT5','BILL_AMT6']
payment_vars = ['PAY_AMT1','PAY_AMT2','PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6']
for i in range(0,len(bill_vars)):
    print(bill_vars[i], payment_vars[i], data.loc[data[bill_vars[i]] < data[payment_vars[i]]].shape[0])

# Feature engineering

In [ ]:
# Categorize age
def categorize_age(x):
    if 16 <= x <= 25:
        return "16_25"
    elif 26 <= x <= 35:
        return "26_35"
    elif 36 <= x <= 45:
        return "36_45"
    elif 46 <= x <= 55:
        return "46_55"
    elif x >= 56:
        return "56_xx"
    else:
        return "na"
    
# Encode late payments
def encode_late_payments(x):
    if x == -2:
        pass # This signifies no bill and will be coded as a binary
    if x == -1:
        pass # This signifies the bill is duly paid
    if x >= 0:
        return x

def sum_late_months(x):
    late_vars = ['PAY_0','PAY_2','PAY_3','PAY_4','PAY_5','PAY_6']
    s = 0
    for l in late_vars:
        if x[l] >= 0:
            s += x[l]
    return s

def sum_duly_paid(x):
    late_vars = ['PAY_0','PAY_2','PAY_3','PAY_4','PAY_5','PAY_6']
    s = 0
    for l in late_vars:
        if x[l] == -1:
            s += 1
    return s

def sum_no_bill(x):
    late_vars = ['PAY_0','PAY_2','PAY_3','PAY_4','PAY_5','PAY_6']
    s = 0
    for l in late_vars:
        if x[l] == -2:
            s += 1
    return s

def sum_bill_amounts(x):
    bill_vars = ['BILL_AMT1','BILL_AMT2','BILL_AMT3','BILL_AMT4','BILL_AMT5','BILL_AMT6']
    s = 0
    for b in bill_vars:
        s += x[b]
    return s

def sum_neg_bills(x):
    bill_vars = ['BILL_AMT1','BILL_AMT2','BILL_AMT3','BILL_AMT4','BILL_AMT5','BILL_AMT6']
    s = 0
    for b in bill_vars:
        if x[b] < 0:
            s += 1
    return s

def sum_payment_amounts(x):
    payment_vars = ['PAY_AMT1','PAY_AMT2','PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6']
    s = 0
    for p in payment_vars:
        s += x[p]
    return s

In [ ]:
X = pd.DataFrame()

# Continuous variables
X['credit_limit'] = data['LIMIT_BAL']
X['age'] = data['AGE']

# Binary variables
X['is_male'] = data['SEX'].apply(lambda x: 1 if x == 1 else 0)

# One-hot encoded
# Age
age_one_hot = pd.get_dummies(data['AGE'].apply(categorize_age),drop_first=True)
X = pd.merge(X,age_one_hot.add_prefix('age_'), how='left',left_index=True, right_index=True)

# Marriage
marital_status = {1: "married", 2: "single", 3: "other", 0: "na_0"}
marriage_one_hot = pd.get_dummies(data['MARRIAGE'].replace(marital_status),drop_first=True)
X = pd.merge(X,marriage_one_hot.add_prefix('marriage_'), how='left',left_index=True, right_index=True)

# Education
education = {1: "graduate_school", 2: "unversity", 3: "high_school", 4: "other", 5: "na_5", 6: "na_6", 0: "na_0"}
education_one_hot = pd.get_dummies(data['EDUCATION'].replace(education),drop_first=True)
X = pd.merge(X,education_one_hot.add_prefix('education_'), how='left',left_index=True, right_index=True)

# Bill late status
late_vars = ['PAY_0','PAY_2','PAY_3','PAY_4','PAY_5','PAY_6']
late_month_mapping = {
    'PAY_0': 'sep',
    'PAY_2': 'aug',
    'PAY_3': 'jul',
    'PAY_4': 'jun',
    'PAY_5': 'may',
    'PAY_6': 'apr'
}

for l in late_vars:
    X[late_month_mapping[l] + '_months_late'] = data[l]
    X[late_month_mapping[l] + '_duly_paid'] = data[l].apply(lambda x: 1 if x == -1 else 0)
    X[late_month_mapping[l] + '_no_bill'] = data[l].apply(lambda x: 1 if x == -2 else 0)

X['total_months_late'] = data.apply(sum_late_months, axis = 1)
X['total_duly_paid'] = data.apply(sum_duly_paid, axis = 1)
X['total_no_bill'] = data.apply(sum_no_bill, axis = 1)

# Bill amounts
bill_vars = ['BILL_AMT1','BILL_AMT2','BILL_AMT3','BILL_AMT4','BILL_AMT5','BILL_AMT6']
bill_month_mapping = {
    'BILL_AMT1': 'sep',
    'BILL_AMT2': 'aug',
    'BILL_AMT3': 'jul',
    'BILL_AMT4': 'jun',
    'BILL_AMT5': 'may',
    'BILL_AMT6': 'apr'
}
for b in bill_vars:
    X[bill_month_mapping[b] + '_bill_amount'] = data[b]
    # Negative bills?
    X[bill_month_mapping[b] + '_neg_bill'] = data[b].apply(lambda x: 1 if x < 0 else 0)
    
X['total_bill_amount'] = data.apply(sum_bill_amounts, axis=1)
X['total_neg_bills'] = data.apply(sum_neg_bills, axis=1)

# Paymount amounts
payment_vars = ['PAY_AMT1','PAY_AMT2','PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6']
payment_month_mapping = {
    'PAY_AMT1': 'sep',
    'PAY_AMT2': 'aug',
    'PAY_AMT3': 'jul',
    'PAY_AMT4': 'jun',
    'PAY_AMT5': 'may',
    'PAY_AMT6': 'apr'
}
for p in payment_vars:
    X[payment_month_mapping[p] + '_payment_amount'] = data[p]
    # Negative bills?
    
X['total_payment_amount'] = data.apply(sum_payment_amounts, axis=1)

X.head()

In [ ]:
X.shape

## Target

In [ ]:
y = data['default payment next month']

## Save

In [ ]:
# Save the csvs
X.to_csv("matrices/loans/X.csv")
y.to_csv("matrices/loans/y.csv")